# Serial DIC

In [ ]:
%%writefile codeSerial.cpp
#include <iostream>
#include <vector>
#include <set>
#include <map>
#include <algorithm>
#include <fstream>
#include <regex>
#include <chrono>

using namespace std;
using namespace std::chrono;

using Transazione = set<string>;
using TransactionDB = vector<Transazione>;

TransactionDB read_csv_and_save_binary(const string& filename, const string& binary_filename) {
    cout << "Lettura e Salvataggio in file binario ..." << endl;
    ifstream infile(filename);
    TransactionDB dataset;
    string line;
    regex pattern("'([^']*)'");

    // Leggi il file CSV
    getline(infile, line);
    while (getline(infile, line)) {
        Transazione transaction;
        sregex_iterator iter(line.begin(), line.end(), pattern);
        sregex_iterator end;

        for (; iter != end; ++iter) {
            string item = (*iter)[1].str();
            transform(item.begin(), item.end(), item.begin(), ::tolower); // conversione in lowercase
            transaction.insert(item);
        }

        dataset.push_back(transaction);
    }

    ofstream outfile(binary_filename, ios::binary);
    for (const auto& transaction : dataset) {
        size_t size = transaction.size();
        outfile.write(reinterpret_cast<const char*>(&size), sizeof(size));
        for (const auto& item : transaction) {
            size_t length = item.size();
            outfile.write(reinterpret_cast<const char*>(&length), sizeof(length));
            outfile.write(item.c_str(), length);
        }
    }
    cout << "File CSV letto e salvato in formato binario!" << endl;
    return dataset;
}

TransactionDB read_binary(const string& binary_filename) {
    ifstream infile(binary_filename, ios::binary);
    TransactionDB dataset;

    while (infile) {
        size_t size;
        infile.read(reinterpret_cast<char*>(&size), sizeof(size));
        if (infile.eof()) break;

        Transazione transaction;
        for (size_t i = 0; i < size; ++i) {
            size_t length;
            infile.read(reinterpret_cast<char*>(&length), sizeof(length));
            string item(length, '\0');
            infile.read(&item[0], length);
            transform(item.begin(), item.end(), item.begin(), ::tolower);
            transaction.insert(item);
        }
        dataset.push_back(transaction);
    }

    cout << "File binario letto!" << endl;
    return dataset;
}

TransactionDB read_csv(const string& filename) {
    ifstream infile(filename);
    TransactionDB dataset;
    string line;
    regex pattern("'([^']*)'");

    while (getline(infile, line)) {
        Transazione transaction;
        sregex_iterator iter(line.begin(), line.end(), pattern);
        sregex_iterator end;

        for (; iter != end; ++iter) {
            string item = (*iter)[1].str();
            transform(item.begin(), item.end(), item.begin(), ::tolower);
            transaction.insert(item);
        }

        dataset.push_back(transaction);
    }
    cout << "Lettura file completata!" << endl;
    return dataset;
}

// Funzioni ausiliarie
bool isSubset(const set<string>& subset, const set<string>& superset) {
    return includes(superset.begin(), superset.end(), subset.begin(), subset.end());
}

// Struttura per rappresentare gli itemsets
struct Itemset {
    set<string> items;    // Elementi dell'itemset
    int support;       // Conteggio del supporto
    int transazioni;

    Itemset() : support(0), transazioni(0) {}
    Itemset(set<string> items) : items(items), support(0), transazioni(0) {}

    bool operator<(const Itemset& other) const {
        return items < other.items; // Definisce l'ordinamento nel set
    }
};

// Funzione principale: SERIALDIC
set<Itemset> SerialDIC(const vector<set<string>>& database, int minsup, int M) {
    // Inizializzazione
    set<Itemset> solidBox;     // Itemsets frequenti confermati
    set<Itemset> dashedBox;    // Itemsets candidati frequenti
    set<Itemset> dashedCircle; // Itemsets candidati infrequenti
    set<Itemset> solidCircle;  // Itemsets infrequenti confermati

    // Inizializza tutti gli itemset di 1 elemento
    map<string, Itemset> candidates;
    for (const auto& transaction : database) {
        for (string item : transaction) {
            if (candidates.find(item) == candidates.end()) {
                candidates[item] = Itemset({ item });
            }
        }
    }
    for (const auto& it : candidates) {
        dashedCircle.insert(it.second);
    }

    size_t start = 0;
    // Ciclo principale
    while (!dashedCircle.empty() || !dashedBox.empty()) {
        // Scansione del database in blocchi
        size_t end = min(start + M, database.size());
        for (size_t t = start; t < end; ++t) {
            const auto& transaction = database[t];

            // Conteggio del supporto per gli itemsets candidati
            set<Itemset> updatedDashedCircle;
            for (auto itemset : dashedCircle) {  // Copia l'itemset per modificarlo
                if (isSubset(itemset.items, transaction)) {
                    itemset.support++;
                }
                itemset.transazioni++;
                updatedDashedCircle.insert(itemset); // Reinserisce l'elemento aggiornato
            }
            dashedCircle = updatedDashedCircle; // Sostituisce il set originale con quello aggiornato

            set<Itemset> updatedDashedBox;
            for (auto itemset : dashedBox) {  // Stesso principio per dashedBox
                if (isSubset(itemset.items, transaction)) {
                    itemset.support++;
                }
                itemset.transazioni++;
                updatedDashedBox.insert(itemset);
            }
            dashedBox = updatedDashedBox;
        }

        // Gestione degli itemsets completati
        set<Itemset> newDashedCircle;
        for (auto& itemset : dashedCircle) {
            if (itemset.support >= minsup) {
                dashedBox.insert(itemset); // Promuovi a DashedBox
                for (const auto& it : candidates) {
                    Itemset single = it.second;
                    set<string> combined;
                    set_union(itemset.items.begin(), itemset.items.end(),
                        single.items.begin(), single.items.end(),
                        inserter(combined, combined.begin()));

                    if (combined.size() == itemset.items.size())
                        continue;

                    bool valid = true;
                    for (string item : combined) {
                        set<string> subset = combined;
                        subset.erase(item);
                        if (!any_of(dashedBox.begin(), dashedBox.end(), [&](const Itemset& is) { return is.items == subset; }) &&
                            !any_of(solidBox.begin(), solidBox.end(), [&](const Itemset& is) { return is.items == subset; })) {
                            valid = false;
                            break;
                        }
                    }
                    if (valid) {
                        newDashedCircle.insert(Itemset(combined));
                    }
                }
            }
            else {
                newDashedCircle.insert(itemset); // Mantieni in DashedCircle
            }
        }
        dashedCircle = newDashedCircle;

        set<Itemset> newDashedCircleFiltered;
        for (auto& itemset : dashedCircle) {
            if (itemset.transazioni == database.size()) {
                solidCircle.insert(itemset);
            }
            else {
                newDashedCircleFiltered.insert(itemset);
            }
        }
        dashedCircle = newDashedCircleFiltered;

        set<Itemset> newDashedBox;
        for (auto& itemset : dashedBox) {
            if (itemset.transazioni == database.size()) {
                solidBox.insert(itemset);
            }
            else {
                newDashedBox.insert(itemset);
            }
        }
        dashedBox = newDashedBox;

        if (end == database.size())
            start = 0;
        else
            start = end;
    }

    return solidBox; // Ritorna gli itemsets frequenti
}

int main() {
    vector<set<string>> database = read_csv("/content/input_generated_10000_10_20.csv");

    int minsup = 1000; // Supporto minimo
    int M = 5000;      // Dimensione del blocco

    auto start = steady_clock::now();
    set<Itemset> frequentItemsets = SerialDIC(database, minsup, M);
    auto stop = steady_clock::now();
    std::chrono::duration<double> duration = stop - start;
    cout << "Tempo di esecuzione Serial DIC: " << duration.count() << " s" << endl;

    ofstream outfile("/content/result_input_generated_10000_10_20.txt");
        for (const auto& result : frequentItemsets) {
            outfile << "Frequent itemset: ";
            for (const auto& item : result.items) {
                outfile << item << " ";
            }
            outfile << "Support: " << result.support << endl;

    }
    outfile.close();

    cout << "Analisi completata, frequent itemsets presenti all'interno del file " << endl;
    return 0;
}


Overwriting codeSerial.cpp


In [ ]:
!g++ -fopenmp -o run_me codeSerial.cpp
!./run_me

Lettura file completata!
Tempo di esecuzione Serial DIC: 304.649 s
Analisi completata, frequent itemsets presenti all'interno del file 


# Parallel DIC, con database di stringhe in input

## Codice:

In [ ]:
%%writefile codeParallel.cpp
#include <iostream>
#include <vector>
#include <set>
#include <map>
#include <algorithm>
#include <fstream>
#include <regex>
#include <cmath>
#include <bitset>
#include <omp.h>
#include <chrono>


using namespace std;
using namespace std::chrono;

using Transazione = set<string>;
using TransactionDB = vector<Transazione>;

TransactionDB read_csv(const string& filename) {
    ifstream infile(filename);
    TransactionDB dataset;
    string line;
    regex pattern("'([^']*)'");

    getline(infile, line);

    while (getline(infile, line)) {
        Transazione transaction;
        sregex_iterator iter(line.begin(), line.end(), pattern);
        sregex_iterator end;

        for (; iter != end; ++iter) {
            string item = (*iter)[1].str();
            transform(item.begin(), item.end(), item.begin(), ::tolower);
            transaction.insert(item);
        }

        dataset.push_back(transaction);
    }
    cout << "Lettura file completata!" << endl;

    return dataset;
}

// Funzioni ausiliarie
bool isSubset(const set<string>& subset, const set<string>& superset) {
    return includes(superset.begin(), superset.end(), subset.begin(), subset.end());
}

enum SHAPE {
    NIL,
    CIRCLE,
    BOX
};

const int m = 81;

// Struttura per rappresentare gli itemsets
struct Itemset {
    std::bitset<m> mask;
    int k;              //numero di items nell'itemset
    SHAPE shape;
    int support;       // Conteggio del supporto
    int stop;

    Itemset() : support(0), stop(0), k(1),shape(NIL) {}
};




pair<map<string,int>, map<int, string>> transformItem(const vector<set<string>>& database) {
    map<int, string> reverse_items;
    map<string, int> items;
    int position = 0;
    for (const auto& transaction : database) {
        for (string item : transaction) {
            if (items.find(item) == items.end()) {
                items[item] = position;
                reverse_items[position] = item;
                position++;
            }
        }
    }

    return { items,reverse_items };
}

vector<std::bitset<m>> transformDB(const vector<set<string>>& database, map<string, int>& items) {

    vector<std::bitset<m>> db;
    for (const auto& transaction : database) {
        std::bitset<m> bitMask;
        for (string item : transaction) {
            int pos = items[item];
            bitMask.set(pos);
        }
        db.push_back(bitMask);
    }

    return db;
}

void firstPass(const vector<std::bitset<m>>& db, vector<Itemset>& solid, vector<Itemset>& dashed,int minsup,map<int,string>reverseItems) {

    //frequent 1-itemsets
    for (auto& itemset : solid) {
        for (const auto& transaction : db) {
            if ((itemset.mask & transaction) == itemset.mask)
                itemset.support++;
        }

        if (itemset.support >= minsup)
            itemset.shape = BOX;
        else
            itemset.shape = CIRCLE;
    }



    //2-itemsets candidates
    for (int i = 0;i < solid.size();i++) {
        if (solid[i].shape == BOX) {
            for (int j = i + 1;j < solid.size();j++) {
                if (solid[j].shape == BOX) {
                    Itemset newItemset;
                    newItemset.k = 2;
                    newItemset.shape = CIRCLE;
                    std::bitset<m> newBitMask = solid[i].mask | solid[j].mask;
                    newItemset.mask = newBitMask;
                    dashed.push_back(newItemset);
                }
            }
        }
    }
}


void countSupport(const vector<std::bitset<m>> db, vector<Itemset>& dashed, int first, int last, int num_of_threads) {
    if (dashed.size() >= num_of_threads) {

        #pragma omp parallel for
        for (int i = 0;i < dashed.size();i++) {
            Itemset& itemset  = dashed[i];
            itemset.stop++;
            for (int j = first;j<=last;j++){
                std::bitset<m> transazione = db[j];
                if ((itemset.mask & transazione) == itemset.mask)
                    itemset.support++;
            }
        }
    }
    else {
        omp_set_nested(true);
        #pragma omp parallel for num_threads(dashed.size())
        for (int i = 0;i < dashed.size();i++) {
            Itemset& itemset = dashed[i];
            itemset.stop++;
            int nThreads = std::ceil(static_cast<double> (num_of_threads) / dashed.size());
            int local_support = 0;

            #pragma omp parallel for reduction(+:local_support) num_threads(nThreads)
            for (int j = first;j <= last;j++) {
                std::bitset<m> transazione = db[j];
                if ((itemset.mask & transazione) == itemset.mask)
                    local_support++;
            }
            itemset.support += local_support;

        }
    }

}

void prune(vector<Itemset>& dashed, int minsup, int stop_max, int num_of_threads, int M) {

    #pragma omp parallel for num_threads(num_of_threads)
    for (int i=0;i < dashed.size();i++) {
        Itemset& itemset = dashed[i];
        if (itemset.shape == CIRCLE) {
            if (itemset.support >= minsup) {
                itemset.shape = BOX;
            }

            else { //fare pruning su tutti gli itemset che non possono mai raggiungere minsup
                int supp_max = itemset.support + M * (stop_max - itemset.stop);
                if (supp_max < minsup) {
                    itemset.shape = NIL;
                    for (auto& itemset_2 : dashed) {
                        if (itemset_2.shape == CIRCLE && ((itemset.mask & itemset_2.mask) == itemset.mask))
                            itemset_2.shape = NIL;
                    }
                }
            }
        }
    }

    //cancella da dashed tutti gli itemset che hanno SHAPE a NIL
    dashed.erase(std::remove_if(dashed.begin(), dashed.end(), [](const Itemset& itemset) { return itemset.shape == NIL;}), dashed.end());

}


#include <unordered_set>
void generateCandidates(vector<Itemset>& dashed, vector<Itemset>& solid) {

    vector<Itemset> newCandidates;
    unordered_set<bitset<m>> existingItemsets; // Set per verificare se un itemset è già presente
    vector<Itemset> frequentItemsets;
    vector<Itemset> single;

    // Inseriamo in `existingItemsets` tutte le maschere già presenti in `dashed` e `solid`
    for (const auto& item : solid) {
        existingItemsets.insert(item.mask);
        if (item.shape == BOX && item.mask.count()==1) {
            single.push_back(item);
        }
    }
    for (const auto& item : dashed) {
        existingItemsets.insert(item.mask);
        if (item.shape == BOX) {
            frequentItemsets.push_back(item);
        }
    }

    // Generazione dei candidati con operazione OR tra itemset BOX
    for (size_t i = 0; i < frequentItemsets.size(); i++) {
        for (size_t j = 0; j < single.size(); j++) {

            bitset<m> newMask = frequentItemsets[i].mask | single[j].mask;

            // Controllo duplicati: se già esiste in `dashed` o `solid`, lo saltiamo
            if (existingItemsets.find(newMask) != existingItemsets.end()) {
                continue;
            }

            // Creazione del nuovo itemset
            Itemset newItemset;
            newItemset.k = newMask.count();

            if (newItemset.k != frequentItemsets[i].k + 1)
                continue;

            newItemset.shape = CIRCLE; // I nuovi candidati partono come "Dashed Circle"
            newItemset.mask = newMask;

            // Verifica che tutti i sottoinsiemi di (k-1) elementi siano già frequenti
            bool valid = true;
            for (size_t k = 0; k < m; ++k) {
                if (newMask.test(k)) { // Bit attivo
                    std::bitset<m> subset = newMask;
                    subset.reset(k); // Rimuove l'elemento corrispondente
                    // Usa la stringa della maschera per controllare duplicati
                    if (existingItemsets.find(subset) == existingItemsets.end()) {
                        valid = false;
                        break;
                    }
                }
            }

            if (valid) {
                newCandidates.push_back(newItemset);
                existingItemsets.insert(newMask); // Aggiungiamo anche nel set per evitare futuri duplicati
            }
        }
    }

    // Aggiungi i nuovi candidati a Dashed Circle (ossia a dashed)
    dashed.insert(dashed.end(), newCandidates.begin(), newCandidates.end());

}

/*
void checkFullPass(vector<Itemset>& dashed, vector<Itemset>& solid,int minsup, int num_of_threads,int stop_max,map<int,string> reverseItems) {

    #pragma omp parallel for num_threads(num_of_threads)
    for (int i = 0;i < dashed.size();i++) {
        Itemset itemset = dashed[i];
        if (itemset.stop == stop_max) {

            if (itemset.support >= minsup)
                itemset.shape = BOX;

            #pragma omp critical
            solid.push_back(itemset);

            dashed[i].shape = NIL;
        }
    }



    dashed.erase(std::remove_if(dashed.begin(), dashed.end(), [](const Itemset& itemset) { return itemset.shape == NIL;}), dashed.end());

}

*/

#pragma omp declare reduction(merge : std::vector<Itemset> : omp_out.insert(omp_out.end(), omp_in.begin(), omp_in.end()))

void checkFullPass(std::vector<Itemset>& dashed, std::vector<Itemset>& solid, int minsup, int num_of_threads, int stop_max) {

    #pragma omp parallel for num_threads(num_of_threads) reduction(merge:solid)
    for (int i = 0; i < dashed.size(); i++) {
        Itemset itemset = dashed[i];
        if (itemset.stop == stop_max) {
            if (itemset.support >= minsup)
                itemset.shape = BOX;

            solid.push_back(itemset); // Scrittura parallela sicura grazie alla reduction
            dashed[i].shape = NIL;
        }
    }

    // Eliminazione degli elementi marcati come NIL
    dashed.erase(std::remove_if(dashed.begin(), dashed.end(),
        [](const Itemset& itemset) { return itemset.shape == NIL; }), dashed.end());
}

// Funzione principale: ParallelDIC
vector<Itemset> ParallelDIC(const vector<set<string>>& database, map<string, int>& items, int minsup, int M, int num_threads, map<int, string> reverseItems){

    //trasformazione dell'input
    vector<std::bitset<m>> db = transformDB(database, items);

    //definzione strutture
    vector<Itemset> dashed;
    vector<Itemset> solid;

    //definizione 1-itemsets
    for (auto& item : items) {
        Itemset itemset;
        std::bitset<m> mask;
        mask.set(item.second);
        itemset.mask = mask;
        solid.push_back(itemset);

    }

    //definizione parametri algoritmo
    int k = 1;
    int stop = 0;

    int stop_max;
    if (db.size() % M == 0)
        stop_max = db.size() / M;
    else
       stop_max = int((db.size() + M - 1) / M);

    firstPass(db,solid, dashed,minsup,reverseItems);

    while (!dashed.empty()){


        stop++;

        if (stop > stop_max)
            stop = 1;
        int first = (stop - 1) * M;

        int last = (stop * M) - 1;
        if (last == db.size())
            last = db.size() - 1;

        k++;

        countSupport(db,dashed,first,last, num_threads);
        prune(dashed, minsup, stop_max, num_threads, M);
        generateCandidates(dashed,solid);
        checkFullPass(dashed,solid, minsup,num_threads,stop_max);
    }

    return solid;
}

int main() {

    vector<set<string>> database = read_csv("/content/input_25000.csv");
    pair< map<string, int>, map<int, string>> coppia = transformItem(database);
    map<string, int> items = coppia.first;
    map<int, string>reverseItems = coppia.second;

    int minsup = 5 ;// Supporto minimo
    int M = 25000/2;      // Dimensione del blocco

    int numThreads = 1;
    cout << database.size()<<endl;
    auto start = steady_clock::now();
    vector<Itemset> frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    auto stop = steady_clock::now();
    std::chrono::duration<double> duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;

    numThreads = 2;
    cout << database.size()<<endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;

    numThreads = 4;
    cout << database.size()<<endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;

     numThreads = 8;
    cout << database.size()<<endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;


    numThreads = 16;
    cout << database.size()<<endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;

    numThreads = 32;
    cout << database.size()<<endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;


    ofstream outfile("/content/result_input_25000.txt");
    for (const auto& result : frequentItemsets) {
        if(result.shape==BOX){
            outfile << "Frequent itemset: ";
            for (size_t i = 0; i < m;i++) {
                if (result.mask.test(i))
                {
                    string item = reverseItems[i];
                    outfile << item << " ";
                }
            }
            outfile << "Support: " << result.support << endl;
        }
    }
    outfile.close();

    cout << "Analisi completata, frequent itemsets presenti all'interno del file " << endl;
    return 0;
}


Writing codeParallel.cpp


## Esecuzione

In [ ]:
!g++ -fopenmp -o run_me codeParallel.cpp
!./run_me

Lettura file completata!
25000
Tempo di esecuzione Parallel DIC con  1 =186.75 s
25000
Tempo di esecuzione Parallel DIC con  2 =98.9158 s
25000


# Parallel DIC, con database di interi in input.

In [ ]:
%%writefile codeParallelinteri.cpp
#include <iostream>
#include <vector>
#include <set>
#include <map>
#include <algorithm>
#include <fstream>
#include <regex>
#include <cmath>
#include <bitset>
#include <omp.h>
#include <chrono>
#include <sstream>
#include <string>


using namespace std;
using namespace std::chrono;

using Transazione = set<int>;
using TransactionDB = vector<Transazione>;

TransactionDB read_csv(const string& filename) {
    ifstream infile(filename);
    TransactionDB dataset;
    string line;

    getline(infile, line); // Salta l intestazione

    while (getline(infile, line)) {
        Transazione transaction;
        line.erase(remove(line.begin(), line.end(), '"'), line.end());
        line.erase(remove(line.begin(), line.end(), '['), line.end());
        line.erase(remove(line.begin(), line.end(), ']'), line.end());
        line.erase(remove(line.begin(), line.end(), ' '), line.end());

        istringstream ss(line);
        string num;

        while (getline(ss, num, ',')) {
            num.erase(remove(num.begin(), num.end(), ' '), num.end());
            transaction.insert(stoi(num));
        }

        dataset.push_back(transaction);
    }
    cout << "Lettura file completata!" << endl;

    return dataset;
}


// Funzioni ausiliarie
bool isSubset(const set<string>& subset, const set<string>& superset) {
    return includes(superset.begin(), superset.end(), subset.begin(), subset.end());
}

enum SHAPE {
    NIL,
    CIRCLE,
    BOX
};

const int m = 81;

// Struttura per rappresentare gli itemsets
struct Itemset {
    std::bitset<m> mask;
    int k;              //numero di items nell'itemset
    SHAPE shape;
    int support;       // Conteggio del supporto
    int stop;

    Itemset() : support(0), stop(0), k(1), shape(NIL) {}
};




pair<map<int, int>, map<int, int>> transformItem(const vector<set<int>>& database) {
    map<int, int> reverse_items;
    map<int, int> items;
    int position = 0;
    for (const auto& transaction : database) {
        for (int item : transaction) {
            if (items.find(item) == items.end()) {
                items[item] = position;
                reverse_items[position] = item;
                position++;
            }
        }
    }

    return { items,reverse_items };
}

vector<std::bitset<m>> transformDB(const vector<set<int>>& database, map<int, int>& items) {

    vector<std::bitset<m>> db;
    for (const auto& transaction : database) {
        std::bitset<m> bitMask;
        for (int item : transaction) {
            int pos = items[item];
            bitMask.set(pos);
        }
        db.push_back(bitMask);
    }

    return db;
}

void firstPass(const vector<std::bitset<m>>& db, vector<Itemset>& solid, vector<Itemset>& dashed, int minsup, map<int, int>reverseItems) {

    //frequent 1-itemsets
    for (auto& itemset : solid) {
        for (const auto& transaction : db) {
            if ((itemset.mask & transaction) == itemset.mask)
                itemset.support++;
        }

        if (itemset.support >= minsup)
            itemset.shape = BOX;
        else
            itemset.shape = CIRCLE;
    }



    //2-itemsets candidates
    for (int i = 0;i < solid.size();i++) {
        if (solid[i].shape == BOX) {
            for (int j = i + 1;j < solid.size();j++) {
                if (solid[j].shape == BOX) {
                    Itemset newItemset;
                    newItemset.k = 2;
                    newItemset.shape = CIRCLE;
                    std::bitset<m> newBitMask = solid[i].mask | solid[j].mask;
                    newItemset.mask = newBitMask;
                    dashed.push_back(newItemset);
                }
            }
        }
    }
}


void countSupport(const vector<std::bitset<m>> db, vector<Itemset>& dashed, int first, int last, int num_of_threads) {
    if (dashed.size() >= num_of_threads) {

        #pragma omp parallel for
        for (int i = 0;i < dashed.size();i++) {
            Itemset& itemset = dashed[i];
            itemset.stop++;
            for (int j = first;j <= last;j++) {
                std::bitset<m> transazione = db[j];
                if ((itemset.mask & transazione) == itemset.mask)
                    itemset.support++;
            }
        }
    }
    else {
        omp_set_nested(true);
        #pragma omp parallel for num_threads(dashed.size())
        for (int i = 0;i < dashed.size();i++) {
            Itemset& itemset = dashed[i];
            itemset.stop++;
            int nThreads = std::ceil(static_cast<double> (num_of_threads) / dashed.size());
            int local_support = 0;  // Variabile locale per la riduzione

            #pragma omp parallel for reduction(+:local_support) num_threads(nThreads)
            for (int j = first;j <= last;j++) {
                std::bitset<m> transazione = db[j];
                if ((itemset.mask & transazione) == itemset.mask)
                    local_support++;
            }
            itemset.support += local_support;  // Aggiorna la struttura originale dopo la riduzione

        }
    }

}

void prune(vector<Itemset>& dashed, int minsup, int stop_max, int num_of_threads, int M) {

    #pragma omp parallel for num_threads(num_of_threads)
    for (int i = 0;i < dashed.size();i++) {
        Itemset& itemset = dashed[i];
        if (itemset.shape == CIRCLE) {
            if (itemset.support >= minsup) {
                itemset.shape = BOX;
            }

            else { //fare pruning su tutti gli itemset che non possono mai raggiungere minsup
                int supp_max = itemset.support + M * (stop_max - itemset.stop);
                if (supp_max < minsup) {
                    itemset.shape = NIL;
                    for (auto& itemset_2 : dashed) {
                        if (itemset_2.shape == CIRCLE && ((itemset.mask & itemset_2.mask) == itemset.mask))
                            itemset_2.shape = NIL;
                    }
                }
            }
        }
    }

    //cancella da dashed tutti gli itemset che hanno SHAPE a NIL
    dashed.erase(std::remove_if(dashed.begin(), dashed.end(), [](const Itemset& itemset) { return itemset.shape == NIL;}), dashed.end());

}


#include <unordered_set>
void generateCandidates(vector<Itemset>& dashed, vector<Itemset>& solid) {

    vector<Itemset> newCandidates;
    unordered_set<bitset<m>> existingItemsets; // Set per verificare se un itemset è già presente
    vector<Itemset> frequentItemsets;
    vector<Itemset> single;

    // Inseriamo in `existingItemsets` tutte le maschere già presenti in `dashed` e `solid`
    for (const auto& item : solid) {
        existingItemsets.insert(item.mask);
        if (item.shape == BOX && item.mask.count() == 1) {
            single.push_back(item);
        }
    }
    for (const auto& item : dashed) {
        existingItemsets.insert(item.mask);
        if (item.shape == BOX) {
            frequentItemsets.push_back(item);
        }
    }

    // Generazione dei candidati con operazione OR tra itemset BOX
    for (size_t i = 0; i < frequentItemsets.size(); i++) {
        for (size_t j = 0; j < single.size(); j++) {

            bitset<m> newMask = frequentItemsets[i].mask | single[j].mask;

            // Controllo duplicati: se già esiste in `dashed` o `solid`, lo saltiamo
            if (existingItemsets.find(newMask) != existingItemsets.end()) {
                continue;
            }

            // Creazione del nuovo itemset
            Itemset newItemset;
            newItemset.k = newMask.count();

            if (newItemset.k != frequentItemsets[i].k + 1)
                continue;

            newItemset.shape = CIRCLE; // I nuovi candidati partono come "Dashed Circle"
            newItemset.mask = newMask;

            // Verifica che tutti i sottoinsiemi di (k-1) elementi siano già frequenti
            bool valid = true;
            for (size_t k = 0; k < m; ++k) {
                if (newMask.test(k)) { // Bit attivo
                    std::bitset<m> subset = newMask;
                    subset.reset(k); // Rimuove l'elemento corrispondente
                    // Usa la stringa della maschera per controllare duplicati
                    if (existingItemsets.find(subset) == existingItemsets.end()) {
                        valid = false;
                        break;
                    }
                }
            }

            if (valid) {
                newCandidates.push_back(newItemset);
                existingItemsets.insert(newMask); // Aggiungiamo anche nel set per evitare futuri duplicati
            }
        }
    }

    // Aggiungi i nuovi candidati a Dashed Circle (ossia a dashed)
    dashed.insert(dashed.end(), newCandidates.begin(), newCandidates.end());

}

/*
void checkFullPass(vector<Itemset>& dashed, vector<Itemset>& solid, int minsup, int num_of_threads, int stop_max, map<int, int> reverseItems) {

    #pragma omp parallel for num_threads(num_of_threads)
    for (int i = 0;i < dashed.size();i++) {
        Itemset itemset = dashed[i];
        if (itemset.stop == stop_max) {

            if (itemset.support >= minsup)
                itemset.shape = BOX;

            #pragma omp critical
            solid.push_back(itemset);

            dashed[i].shape = NIL;
        }
    }



    dashed.erase(std::remove_if(dashed.begin(), dashed.end(), [](const Itemset& itemset) { return itemset.shape == NIL;}), dashed.end());

}
*/

#pragma omp declare reduction(merge : std::vector<Itemset> : omp_out.insert(omp_out.end(), omp_in.begin(), omp_in.end()))

void checkFullPass(std::vector<Itemset>& dashed, std::vector<Itemset>& solid, int minsup, int num_of_threads, int stop_max) {

    #pragma omp parallel for num_threads(num_of_threads) reduction(merge:solid)
    for (int i = 0; i < dashed.size(); i++) {
        Itemset itemset = dashed[i];
        if (itemset.stop == stop_max) {
            if (itemset.support >= minsup)
                itemset.shape = BOX;

            solid.push_back(itemset); // Scrittura parallela sicura grazie alla reduction
            dashed[i].shape = NIL;
        }
    }

    // Eliminazione degli elementi marcati come NIL
    dashed.erase(std::remove_if(dashed.begin(), dashed.end(),
        [](const Itemset& itemset) { return itemset.shape == NIL; }), dashed.end());
}

// Funzione principale: ParallelDIC
vector<Itemset> ParallelDIC(const vector<set<int>>& database, map<int, int>& items, int minsup, int M, int num_threads, map<int, int> reverseItems) {

    //trasformazione dell'input
    vector<std::bitset<m>> db = transformDB(database, items);

    //definzione strutture
    vector<Itemset> dashed;
    vector<Itemset> solid;

    //definizione 1-itemsets
    for (auto& item : items) {
        Itemset itemset;
        std::bitset<m> mask;
        mask.set(item.second);
        itemset.mask = mask;
        solid.push_back(itemset);

    }

    //definizione parametri algoritmo
    int k = 1;
    int stop = 0;

    int stop_max;
    if (db.size() % M == 0)
        stop_max = db.size() / M;
    else
        stop_max = int((db.size() + M - 1) / M);

    firstPass(db, solid, dashed, minsup, reverseItems);

    while (!dashed.empty()) {


        stop++;

        if (stop > stop_max)
            stop = 1;
        int first = (stop - 1) * M;

        int last = (stop * M) - 1;
        if (last == db.size())
            last = db.size() - 1;

        k++;

        countSupport(db, dashed, first, last, num_threads);
        prune(dashed, minsup, stop_max, num_threads, M);
        generateCandidates(dashed, solid);
        checkFullPass(dashed, solid, minsup, num_threads, stop_max);
    }

    return solid;
}

int main() {

    vector<set<int>> database = read_csv("/content/input_25000.csv");
    pair< map<int, int>, map<int, int>> coppia = transformItem(database);
    map<int, int> items = coppia.first;
    map<int, int>reverseItems = coppia.second;

    int minsup = 10;// Supporto minimo
    int M = 12500;      // Dimensione del blocco

    int numThreads = 1;
    cout << database.size() << endl;
    auto start = steady_clock::now();
    vector<Itemset> frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    auto stop = steady_clock::now();
    std::chrono::duration<double> duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;

    numThreads = 2;
    cout << database.size() << endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;

    numThreads = 4;
    cout << database.size() << endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;

    numThreads = 8;
    cout << database.size() << endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;


    numThreads = 16;
    cout << database.size() << endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;

    numThreads = 32;
    cout << database.size() << endl;
    start = steady_clock::now();
    frequentItemsets = ParallelDIC(database, items, minsup, M, numThreads, reverseItems);
    stop = steady_clock::now();
    duration = stop - start;
    cout << "Tempo di esecuzione Parallel DIC con  " << numThreads << " =" << duration.count() << " s" << endl;


    ofstream outfile("/content/result_25000.txt");
    for (const auto& result : frequentItemsets) {
        if (result.shape == BOX) {
            outfile << "Frequent itemset: ";
            for (size_t i = 0; i < m;i++) {
                if (result.mask.test(i))
                {
                    int item = reverseItems[i];
                    outfile << item << " ";
                }
            }
            outfile << "Support: " << result.support << endl;
        }
    }
    outfile.close();

    cout << "Analisi completata, frequent itemsets presenti all'interno del file " << endl;
    return 0;
}


Overwriting codeParallelinteri.cpp


In [ ]:
!g++ -fopenmp -o run_me codeParallelinteri.cpp
!./run_me

terminate called after throwing an instance of 'std::invalid_argument'
  what():  stoi
